### Data Cleaning
---
    1. Table `customer_order`

In [1]:
DROP TABLE IF EXISTS #customer_orders
SELECT 
    order_id,
    customer_id,
    pizza_id,
    CASE 
        WHEN exclusions = '' OR exclusions like 'null' THEN NULL
        ELSE exclusions
    END as exclusions,
    CASE 
        WHEN extras = '' OR extras like 'null' THEN NULL
        ELSE extras
    END as extras,
    order_time
INTO #customer_orders
FROM
    customer_orders

(14 rows affected)

Total execution time: 00:00:00.080

    2. Table `runner_orders`

In [2]:
DROP TABLE IF EXISTS #runner_orders
SELECT 
    order_id,
    runner_id,
    CASE 
        WHEN pickup_time like 'null' THEN NULL
        ELSE pickup_time
    END as pickup_time,
    CASE 
        WHEN distance like 'null' THEN NULL
        WHEN distance like '%km' THEN TRIM('km' FROM distance)
        ELSE distance
    END as distance,
    CASE 
        WHEN duration like 'null' THEN NULL
        WHEN duration like '%minutes' THEN TRIM('minutes' FROM duration)
        WHEN duration like '%mins' THEN TRIM('mins' FROM duration)
        WHEN duration like '%minute' THEN TRIM('minute' FROM duration)
        ELSE duration
    END as duration,
    CASE 
        WHEN cancellation like 'null' THEN NULL
        WHEN cancellation ='' THEN NULL
        ELSE cancellation
    END as cancellation
INTO #runner_orders
FROM
    runner_orders;

(10 rows affected)

Total execution time: 00:00:00.040

    3. Changing data type

In [3]:
ALTER TABLE #runner_orders
ALTER COLUMN pickup_time DATETIME

ALTER TABLE #runner_orders
ALTER COLUMN distance FLOAT

ALTER TABLE #runner_orders
ALTER COLUMN duration INT

ALTER TABLE pizza_names
ALTER COLUMN pizza_name VARCHAR(50)

ALTER TABLE pizza_recipes
ALTER COLUMN toppings VARCHAR(50)

ALTER TABLE pizza_toppings
ALTER COLUMN topping_name VARCHAR(50)

Commands completed successfully.

Total execution time: 00:00:00.077

### A. Pizza Metrics
---
    1. How many pizzas were ordered?


In [4]:
SELECT 
    COUNT(*) as Number_pizzas_ordered
FROM
    #customer_orders

(1 row affected)

Total execution time: 00:00:00.059

Number_pizzas_ordered
14


    2. How many unique customer orders were made?

In [5]:
SELECT
    COUNT(DISTINCT order_id) as unique_customer_orders
FROM
    #customer_orders

(1 row affected)

Total execution time: 00:00:00.026

unique_customer_orders
10


    3. How many successful orders were delivered by each runner?

In [6]:
SELECT 
    runner_id,
    COUNT(*) as successful_orders
FROM
    #runner_orders
WHERE 
    distance != 0
GROUP BY runner_id

(3 rows affected)

Total execution time: 00:00:00.065

runner_id,successful_orders
1,4
2,3
3,1


    4. How many of each type of pizza was delivered?

In [7]:
SELECT 
    c.pizza_id,
    COUNT(*) as type_of_pizza_delivered
FROM
    #customer_orders c 
JOIN #runner_orders r on r.order_id = c.order_id
WHERE r.distance != 0
GROUP BY c.pizza_id

(2 rows affected)

Total execution time: 00:00:00.019

pizza_id,type_of_pizza_delivered
1,9
2,3


    5. How many Vegetarian and Meatlovers were ordered by each customer?

In [8]:
SELECT
    c.customer_id,
    p.pizza_name,
    COUNT(*) as number_of_pizzas
FROM 
    #customer_orders c 
JOIN pizza_names p on p.pizza_id = c.pizza_id
GROUP BY c.customer_id, p.pizza_name
ORDER BY c.customer_id

(8 rows affected)

Total execution time: 00:00:00.012

customer_id,pizza_name,number_of_pizzas
101,Meatlovers,2
101,Vegetarian,1
102,Meatlovers,2
102,Vegetarian,1
103,Meatlovers,3
103,Vegetarian,1
104,Meatlovers,3
105,Vegetarian,1


    6. What was the maximum number of pizzas delivered in a single order?

In [9]:
WITH tbl as(
    SELECT
        c.order_id,
        COUNT(*) as number_of_pizzas
    FROM 
        #customer_orders c 
    JOIN #runner_orders r on r.order_id = c.order_id
    WHERE r.distance != 0
    GROUP BY 
        c.order_id
)
SELECT 
    MAX(number_of_pizzas) as max_pizzas_delivered
FROM
    tbl

(1 row affected)

Total execution time: 00:00:00.006

max_pizzas_delivered
3


    7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?

In [10]:
SELECT
    c.customer_id,
    SUM(
        CASE  
            WHEN c.exclusions = '' and c.extras = '' THEN 1
            ELSE 0
        END
    ) as no_change,
    SUM(
        CASE  
            WHEN c.exclusions != '' OR c.extras != '' THEN 1
            ELSE 0
        END
    ) as at_least_1_change
FROM 
    #customer_orders c 
JOIN #runner_orders r on r.order_id = c.order_id
WHERE r.distance != 0
GROUP BY 
    c.customer_id

(5 rows affected)

Total execution time: 00:00:00.006

customer_id,no_change,at_least_1_change
101,0,0
102,0,0
103,0,3
104,0,2
105,0,1


    8. How many pizzas were delivered that had both exclusions and extras?

In [11]:
SELECT
    COUNT(*) as pizzas_had_both_exclusions_and_extras
FROM 
    #customer_orders c 
JOIN #runner_orders r on r.order_id = c.order_id
WHERE r.distance != 0 AND c.exclusions != '' AND c.extras != ''

(1 row affected)

Total execution time: 00:00:00.020

pizzas_had_both_exclusions_and_extras
1


    9. What was the total volume of pizzas ordered for each hour of the day?

In [12]:
SELECT
    DATEPART(HOUR, order_time) as hour_of_day,
    COUNT(*) as number_of_pizzas
FROM 
    #customer_orders c 
GROUP BY DATEPART(HOUR, order_time)

(6 rows affected)

Total execution time: 00:00:00.014

hour_of_day,number_of_pizzas
11,1
13,3
18,3
19,1
21,3
23,3


    10. What was the volume of orders for each day of the week?

In [13]:
SELECT
  DATENAME(WEEKDAY, order_time) as day_of_week,
  COUNT(*) as number_of_pizzas
FROM
  #customer_orders
GROUP BY DATENAME(WEEKDAY, order_time)
ORDER BY day_of_week

(4 rows affected)

Total execution time: 00:00:00.012

day_of_week,number_of_pizzas
Friday,1
Saturday,5
Thursday,3
Wednesday,5


### B. Runner and Customer Experience
---
    1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

In [14]:
SET DATEFIRST 1;
SELECT
  DATEPART(WEEK, registration_date) as week,
  COUNT(*) as runners_signup
FROM 
  runners
GROUP BY DATEPART(WEEK, registration_date);

(3 rows affected)

Total execution time: 00:00:00.009

week,runners_signup
1,2
2,1
3,1


    2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

In [15]:
WITH tbl as(
  SELECT
    r.runner_id,
    c.order_id,
    c.order_time,
    r.pickup_time,
    CAST(DATEDIFF( minute, order_time, pickup_time) AS FLOAT) as pickup_minutes
  FROM 
    #runner_orders r
  JOIN #customer_orders c on c.order_id = r.order_id
  WHERE r.distance != 0
  GROUP BY r.runner_id, c.order_id, c.order_time, r.pickup_time
)
SELECT 
  runner_id,
  ROUND(AVG(pickup_minutes),2) as avg_pickup_minutes
FROM
  tbl
GROUP BY runner_id;

(3 rows affected)

Total execution time: 00:00:00.088

runner_id,avg_pickup_minutes
1,14.25
2,20.33
3,10


    3. Is there any relationship between the number of pizzas and how long the order takes to prepare?

In [16]:
WITH tbl as(
  SELECT
    c.order_id,
    CAST(DATEDIFF(MINUTE, c.order_time, r.pickup_time) as float) as pickup_minutes,
    COUNT(*) as number_of_pizzas_in_a_order
  FROM 
    #customer_orders c
  JOIN #runner_orders r on r.order_id = c.order_id
  WHERE r.distance != 0
  GROUP BY c.order_id, DATEDIFF(MINUTE, c.order_time, r.pickup_time)
)
SELECT
  number_of_pizzas_in_a_order,
  AVG(pickup_minutes) as avg_time_per_order,
  (AVG(pickup_minutes)/ number_of_pizzas_in_a_order) AS avg_time_per_pizza
FROM
  tbl
GROUP BY number_of_pizzas_in_a_order;

(3 rows affected)

Total execution time: 00:00:01.013

number_of_pizzas_in_a_order,avg_time_per_order,avg_time_per_pizza
1,12.2,12.2
2,18.5,9.25
3,30,10


    4. What was the average distance travelled for each customer?

In [17]:
WITH tbl AS(
  SELECT 
    c.customer_id,
    r.distance
  FROM 
    #customer_orders c
  JOIN #runner_orders r on r.order_id = c.order_id
  WHERE r.distance != 0
  GROUP BY c.customer_id, r.distance
)
SELECT
  customer_id,
  AVG(distance) as avg_distance_travelled
FROM
  tbl
GROUP BY customer_id

(5 rows affected)

Total execution time: 00:00:00.014

customer_id,avg_distance_travelled
101,20
102,18.4
103,23.4
104,10
105,25


    5. What was the difference between the longest and shortest delivery times for all orders?

In [18]:
SELECT
  MAX(duration) - MIN(duration) as biggest_delivery_time_difference
FROM 
  #runner_orders 
WHERE distance != 0

(1 row affected)

Total execution time: 00:00:00.008

biggest_delivery_time_difference
30


    6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

In [19]:
SELECT
  runner_id,
  order_id,
  ROUND(AVG(distance*60/duration),2) as average_speed_each_delivery
FROM 
  #runner_orders r
WHERE distance != 0
GROUP BY runner_id, order_id
ORDER BY runner_id

(8 rows affected)

Total execution time: 00:00:00.020

runner_id,order_id,average_speed_each_delivery
1,1,37.5
1,2,44.44
1,3,40.2
1,10,60
2,4,35.1
2,7,60
2,8,93.6
3,5,40


    7. What is the successful delivery percentage for each runner?

In [20]:
SELECT
  runner_id,
  100 * 
  SUM(
    CASE
      WHEN distance != 0 THEN 1
      ELSE 0
    END
  ) / 
  COUNT(*) as successful_delivery_percentage
FROM 
  #runner_orders 
GROUP BY runner_id;

(3 rows affected)

Total execution time: 00:00:00.016

runner_id,successful_delivery_percentage
1,100
2,75
3,50


### C. Ingredient Optimization
---
### Data cleaning for this section
    1. Table `pizza_recipes`

In [21]:
DROP TABLE IF EXISTS #pizza_recipes;
SELECT pizza_id, 
        TRIM(topping_id.value) as topping_id,
        topping_name
INTO #pizza_recipes
FROM pizza_recipes p
CROSS APPLY string_split(p.toppings, ',') as topping_id
INNER JOIN pizza_toppings p2 ON TRIM(topping_id.value) = p2.topping_id

(14 rows affected)

Total execution time: 00:00:00.007

    2. Table `#customer_orders`

In [22]:
ALTER TABLE #customer_orders
ADD record_id INT IDENTITY(1,1)

Commands completed successfully.

Total execution time: 00:00:00.004

    3. New tables `#extras` & `exclusions`

Table `#extras`

In [23]:
DROP TABLE IF EXISTS #extras
SELECT		
      c.record_id,
      TRIM(e.value) AS topping_id
INTO #extras
FROM #customer_orders as c
	    CROSS APPLY string_split(c.extras, ',') as e;

(6 rows affected)

Total execution time: 00:00:00.003

Table `#exclusions`

In [24]:
DROP TABLE IF EXISTS #exclusions
SELECT	c.record_id,
	      TRIM(e.value) AS topping_id
INTO #exclusions
FROM #customer_orders as c
	    CROSS APPLY string_split(c.exclusions, ',') as e;

(6 rows affected)

Total execution time: 00:00:00.003

### Question

    1. What are the standard ingredients for each pizza?

In [25]:
WITH tbl AS (
              SELECT pizza_id, 
                      p1.topping_name
              FROM #pizza_recipes p1
              INNER JOIN pizza_toppings p2 
              ON p1.topping_id = p2.topping_id
)
SELECT pizza_id, String_agg(topping_name,',') as Standard_toppings
FROM tbl
GROUP BY pizza_id;

(2 rows affected)

Total execution time: 00:00:00.020

pizza_id,Standard_toppings
1,"Bacon,BBQ Sauce,Beef,Cheese,Chicken,Mushrooms,Pepperoni,Salami"
2,"Cheese,Mushrooms,Onions,Peppers,Tomatoes,Tomato Sauce"


    2. What was the most commonly added extra?

In [26]:
WITH CTE AS (
    SELECT 
        pizza_id,
        topping_type,
        topping
    FROM (
        SELECT 
            pizza_id, 
            CAST(SUBSTRING(extras, 1,1) AS INT) AS topping_1, 
            CAST(SUBSTRING(extras,3,3) AS INT) as topping_2
        FROM #customer_orders
        WHERE extras is not null
        ) p 
    UNPIVOT (topping for topping_type in (topping_1,topping_2)) as unpvt
)
SELECT 
    Topping, 
    topping_name, 
    COUNT(topping) AS Extra_Topping_Time
FROM CTE c
JOIN pizza_toppings p ON c.topping = p.topping_id
WHERE topping != 0
GROUP BY topping,topping_name;

(3 rows affected)

Total execution time: 00:00:26.046

Topping,topping_name,Extra_Topping_Time
1,Bacon,4
4,Cheese,1
5,Chicken,1


    3. What was the most common exclusion?

In [27]:
WITH CTE AS (
    SELECT 
        pizza_id,
        topping_type,
        topping
    FROM (
        SELECT pizza_id, 
            CAST(SUBSTRING(exclusions, 1,1) AS INT) AS exclusions_1, 
            CAST(SUBSTRING(exclusions,3,3) AS INT) as exclusions_2
        FROM #customer_orders
        WHERE exclusions is not null
    ) p 
    UNPIVOT (topping for topping_type in (exclusions_1,exclusions_2)) as unpvt
)
SELECT 
    Topping, 
    topping_name,
    count(topping) AS exclusions_Topping_Time
FROM CTE c
JOIN pizza_toppings p ON c.topping = p.topping_id 
WHERE topping != 0
GROUP BY topping,topping_name
ORDER BY exclusions_Topping_Time DESC;

(3 rows affected)

Total execution time: 00:00:00.033

Topping,topping_name,exclusions_Topping_Time
4,Cheese,4
6,Mushrooms,1
2,BBQ Sauce,1


    4. Generate an order item for each record in the customers_orders table in the format of one of the following:
    - Meat Lovers
    - Meat Lovers - Exclude Beef
    - Meat Lovers - Extra Bacon
    - Meat Lovers - Exclude Cheese, Bacon - Extra Mushroom, Peppers

In [28]:
WITH extras_cte AS (
                    SELECT 
                      record_id,
                      'Extra ' + STRING_AGG(t.topping_name, ', ') as record_options
                    FROM #extras e,
                         pizza_toppings t
                    WHERE e.topping_id = t.topping_id
                    GROUP BY record_id
                    ),
exclusions_cte AS   (
                    SELECT 
                      record_id,
                      'Exclude ' + STRING_AGG(t.topping_name, ', ') as record_options
                    FROM #exclusions e,
                         pizza_toppings t
                    WHERE e.topping_id = t.topping_id
                    GROUP BY record_id
                  ),
union_cte AS      (
                    SELECT * FROM extras_cte
                    UNION
                    SELECT * FROM exclusions_cte
                  )

SELECT c.record_id, 
        c.order_id,
        CONCAT_WS(' - ', p.pizza_name, STRING_AGG(cte.record_options, ' - ')) as pizza_and_topping
FROM #customer_orders c
JOIN pizza_names p ON c.pizza_id = p.pizza_id
LEFT JOIN union_cte cte ON c.record_id = cte.record_id
GROUP BY
	c.record_id,
	p.pizza_name,
  c.order_id
ORDER BY 1;

(14 rows affected)

Total execution time: 00:00:00.762

record_id,order_id,pizza_and_topping
1,1,Meatlovers
2,2,Meatlovers
3,3,Meatlovers
4,3,Vegetarian
5,4,Meatlovers - Exclude Cheese
6,4,Meatlovers - Exclude Cheese
7,4,Vegetarian - Exclude Cheese
8,5,Meatlovers - Extra Bacon
9,6,Vegetarian
10,7,Vegetarian - Extra Bacon


    5. Generate an alphabetically ordered comma separated ingredient list for each pizza order from the customer_orders table and add a 2x in front of any relevant ingredients
    - For example: "Meat Lovers: 2xBacon, Beef, ... , Salami"

In [29]:
WITH INGREDIENT_CTE AS (SELECT record_id,
                                pizza_name,
                                CASE WHEN p1.topping_id in (
                                                  SELECT topping_id
                                                  FROM #extras e
                                                  WHERE C.record_id = e.record_id
                                                 ) 
                                      THEN '2x' + p1.topping_name
                                      ELSE p1.topping_name
                                      END AS topping
                        FROM #customer_orders c 
                        JOIN pizza_names p2 ON c.pizza_id = p2.pizza_id
                        JOIN #pizza_recipes p1 ON c.pizza_id = p1.pizza_id
                        WHERE p1.topping_id NOT IN (SELECT topping_id 
                                                 FROM #exclusions e 
                                                 WHERE e.record_id = c.record_id)
                      )

SELECT record_id, 
      CONCAT(pizza_name +':' ,STRING_AGG(topping, ',' ) WITHIN GROUP (ORDER BY topping ASC)) AS ingredient_list
FROM INGREDIENT_CTE
GROUP BY  record_id,pizza_name
ORDER BY 1;

(14 rows affected)

Total execution time: 00:00:00.078

record_id,ingredient_list
1,"Meatlovers:Bacon,BBQ Sauce,Beef,Cheese,Chicken,Mushrooms,Pepperoni,Salami"
2,"Meatlovers:Bacon,BBQ Sauce,Beef,Cheese,Chicken,Mushrooms,Pepperoni,Salami"
3,"Meatlovers:Bacon,BBQ Sauce,Beef,Cheese,Chicken,Mushrooms,Pepperoni,Salami"
4,"Vegetarian:Cheese,Mushrooms,Onions,Peppers,Tomato Sauce,Tomatoes"
5,"Meatlovers:Bacon,BBQ Sauce,Beef,Chicken,Mushrooms,Pepperoni,Salami"
6,"Meatlovers:Bacon,BBQ Sauce,Beef,Chicken,Mushrooms,Pepperoni,Salami"
7,"Vegetarian:Mushrooms,Onions,Peppers,Tomato Sauce,Tomatoes"
8,"Meatlovers:2xBacon,BBQ Sauce,Beef,Cheese,Chicken,Mushrooms,Pepperoni,Salami"
9,"Vegetarian:Cheese,Mushrooms,Onions,Peppers,Tomato Sauce,Tomatoes"
10,"Vegetarian:Cheese,Mushrooms,Onions,Peppers,Tomato Sauce,Tomatoes"


    6. What is the total quantity of each ingredient used in all delivered pizzas sorted by most frequent first?

In [30]:
WITH INGREDIENT_CTE AS (SELECT record_id,
                                pizza_name, 
                                topping_name,
                                CASE WHEN p1.topping_id in (
                                  SELECT topping_id
                                  FROM #extras e
                                  WHERE C.record_id = e.record_id
                                ) THEN 2
                                ELSE 1
                                END AS times_used_topping
                        FROM #customer_orders c 
                        JOIN pizza_names p2 ON c.pizza_id = p2.pizza_id
                        JOIN #pizza_recipes p1 ON c.pizza_id = p1.pizza_id
                        JOIN #runner_orders r ON c.order_id = r.order_id
                        WHERE p1.topping_id NOT IN (SELECT topping_id 
                                                  FROM #exclusions e 
                                                  WHERE e.record_id = c.record_id) 
                                                  and r.cancellation is NULL
                         )

SELECT topping_name, 
        SUM(times_used_topping) AS times_used_topping
from INGREDIENT_CTE
GROUP BY topping_name
order by times_used_topping desc;

(12 rows affected)

Total execution time: 00:00:00.783

topping_name,times_used_topping
Bacon,11
Mushrooms,11
Cheese,10
Chicken,9
Pepperoni,9
Salami,9
Beef,9
BBQ Sauce,8
Peppers,3
Onions,3


### D. Pricing and Ratings
---
    1. If a Meat Lovers pizza costs $12 and Vegetarian costs $10 and there were no charges for changes - how much money has Pizza Runner made so far if there are no delivery fees?

In [31]:
WITH CTE AS (
    SELECT 
        pizza_id, 
        pizza_name,
        CASE WHEN pizza_name = 'Meatlovers' THEN 12 ELSE 10 END AS pizza_cost
    FROM pizza_names
) 
SELECT SUM(pizza_cost) as total_revenue
FROM #customer_orders c 
JOIN #runner_orders r ON c.order_id = r.order_id
JOIN CTE c2 ON c.pizza_id = c2.pizza_id
WHERE r.cancellation IS NULL;

(1 row affected)

Total execution time: 00:00:00.073

total_revenue
138


    2. What if there was an additional $1 charge for any pizza extras?
    - Add cheese is $1 extra

In [32]:
WITH pizza_cte AS
          (SELECT 
                  (CASE WHEN pizza_id=1 THEN 12
                        WHEN pizza_id = 2 THEN 10
                        END) AS pizza_cost, 
                  c.exclusions,
                  c.extras
          FROM #runner_orders r
          JOIN #customer_orders c ON c.order_id = r.order_id
          WHERE r.cancellation IS  NULL
          )
SELECT 
      SUM(CASE WHEN extras IS NULL THEN pizza_cost
               WHEN DATALENGTH(extras) = 1 THEN pizza_cost + 1
               ELSE pizza_cost + 2
                END ) AS total_earn
FROM pizza_cte;

(1 row affected)

Total execution time: 00:00:00.022

total_earn
142


    3. The Pizza Runner team now wants to add an additional ratings system that allows customers to rate their runner, how would you design an additional table for this new dataset - generate a schema for this new table and insert your own data for ratings for each successful customer order between 1 to 5.

In [33]:
DROP TABLE IF EXISTS ratings
CREATE TABLE ratings 
 (order_id INTEGER,
    rating INTEGER);
INSERT INTO ratings
 (order_id ,rating)
VALUES 
(1,3),
(2,4),
(3,5),
(4,2),
(5,1),
(6,3),
(7,4),
(8,1),
(9,3),
(10,5); 

SELECT * 
from ratings

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.044

order_id,rating
1,3
2,4
3,5
4,2
5,1
6,3
7,4
8,1
9,3
10,5


    4. Using your newly generated table - can you join all of the information together to form a table which has the following information for successful deliveries?
    - customer_id
    - order_id
    - runner_id
    - rating
    - order_time
    - pickup_time
    - Time between order and pickup
    - Delivery duration
    - Average speed
    - Total number of pizzas

In [34]:
SELECT customer_id , 
        c.order_id, 
        runner_id, 
        rating, 
        order_time, 
        pickup_time, 
        datepart( minute,pickup_time - order_time) as Time_order_pickup, 
        r.duration, 
        round(avg(distance/duration*60),2) as avg_Speed, 
        COUNT(pizza_id) AS Pizza_Count
FROM #customer_orders c
LEFT JOIN #runner_orders r ON c.order_id = r.order_id 
LEFT JOIN ratings r2 ON c.order_id = r2.order_id
WHERE r.cancellation is NULL
GROUP BY customer_id , c.order_id, runner_id, rating, order_time, pickup_time, datepart( minute,pickup_time - order_time) , r.duration
ORDER BY c.customer_id;

(8 rows affected)

Total execution time: 00:00:00.069

customer_id,order_id,runner_id,rating,order_time,pickup_time,Time_order_pickup,duration,avg_Speed,Pizza_Count
101,1,1,3,2020-01-01 18:05:02.000,2020-01-01 18:15:34.000,10,32,37.5,1
101,2,1,4,2020-01-01 19:00:52.000,2020-01-01 19:10:54.000,10,27,44.44,1
102,3,1,5,2020-01-02 23:51:23.000,2020-01-03 00:12:37.000,21,20,40.2,2
102,8,2,1,2020-01-09 23:54:33.000,2020-01-10 00:15:02.000,20,15,93.6,1
103,4,2,2,2020-01-04 13:23:46.000,2020-01-04 13:53:03.000,29,40,35.1,3
104,5,3,1,2020-01-08 21:00:29.000,2020-01-08 21:10:57.000,10,15,40,1
104,10,1,5,2020-01-11 18:34:49.000,2020-01-11 18:50:20.000,15,10,60,2
105,7,2,4,2020-01-08 21:20:29.000,2020-01-08 21:30:45.000,10,25,60,1


    5. If a Meat Lovers pizza was $12 and Vegetarian $10 fixed prices with no cost for extras and each runner is paid $0.30 per kilometre traveled - how much money does Pizza Runner have left over after these deliveries?

In [35]:
WITH CTE AS (SELECT c.order_id,
                    SUM(CASE WHEN pizza_name = 'Meatlovers' THEN 12
                          ELSE 10 END) AS pizza_cost
             FROM pizza_names p
             JOIN #customer_orders c ON p.pizza_id =c.pizza_id
             GROUP BY c.order_id) 

SELECT SUM(pizza_cost) AS revenue, 
       SUM(distance) *0.3 as total_cost,
       SUM(pizza_cost) - SUM(distance)*0.3 as profit
FROM #runner_orders r 
JOIN CTE c ON R.order_id =C.order_id
WHERE r.cancellation is NULL

(1 row affected)

Total execution time: 00:00:00.018

revenue,total_cost,profit
138,43.559999999999995,94.44
